# Basic Authentication

Many OPeNDAP servers require authentication. [PyDAP](https://github.com/pydap/pydap) uses Python's [requests](https://requests.readthedocs.io/en/latest/) library to fetch data across the web, and it also handle two general forms of authentication:


1. `username/password`.
2. `tokens`

### Requirement

- A `.netrc` file on your local system, to hold your authentication credentials(on Window machines, it is called `_netrc`).

```{note}
Below we will demonstrate how to create a `.netrc` file in your local system.
```

We strongly recommend having a static, local `~/.netrc` file that holds all authentication credentials, since [requests](https://requests.readthedocs.io/en/latest/) can handle authentication from a `.netrc` file for you, without you the user having to do any extra step. This is, `pydap` can "discover" your OPeNDAP authentication (username/password) credentials efficiently, if these are properly defined on a `.netrc` file (`_netrc` on Windows machines).

While not all OPeNDAP servers are set up to authenticate via `token`, the vast majority of NASA's Hyrax servers are, and the token is created via your Earth Data Login (EDL) account. Betewen the two approaches above, `token authentication` is generally recommended thoughout the documentation as it can avoid many (but not all) redirects.


```{note}
For a broader introduction to authentication, check the `OPeNDAP` documentation on [Authentication for DAP Clients](https://opendap.github.io/documentation/tutorials/ClientAuthentication.html), and [Client Authentication With EDLTokens](https://opendap.github.io/documentation/tutorials/ClientAuthenticationWithEDLTokens.html).
```


### Let's access some remote data!

In [ ]:
from pydap.client import open_url
from pydap.net import create_session

In [ ]:
dataset_url = "https://opendap.earthdata.nasa.gov/providers/POCLOUD/collections/ECCO%20Ocean%20Temperature%20and%20Salinity%20-%20Monthly%20Mean%200.5%20Degree%20(Version%204%20Release%204)/granules/OCEAN_TEMPERATURE_SALINITY_mon_mean_2017-12_ECCO_V4r4_latlon_0p50deg"

## <font size="5"><span style='color:#0066cc'> **username / password**<font size="3">

Nothing is needed, other than initiate a `requests.Session` object. `pydap.net` has a function that can initiate such a session along with some additional parameters. This is:


In [ ]:
my_session = create_session() # default requests.session() object. 
print(my_session)

or if you want to cache the session to avoid repeated downloads:

In [ ]:
my_session = create_session(use_cache=True) # default requests_cache.session() object. 
print(my_session)

```{note}
both [requests](https://requests.readthedocs.io/en/latest/) and [requests_cache](https://requests-cache.readthedocs.io/en/stable/) can recover and handle authentication via `.netrc`
```

### Now, we open the remote dataset

Now, all that is needed is:

```python
ds = open_url(dataset_url, session=my_session, protocol="dap4")
```

The (default) requests session will recover the .netrc credentials, and no additional information is needed.


## <font size="5"><span style='color:#0066cc'> **Tokens**<font size="3">

This is another approach  that is supported by various institutions, and will be used throughout the documentation. To enable token authentication, you must first make sure you have a valid (unexpired) token, and if not create a new one. For example, see this [EDL resource](https://opendap.github.io/documentation/tutorials/ClientAuthenticationWithEDLTokens.html) that is specific to NASA. 

To improve the user experience, `pydap.net.create_session` can take extra session arguments, such as the `token` to:

- Create a `requests.session` with token information on its headers.
- Disable automatic discovery of the `.netrc` file.




In [ ]:
session_extra = {"token": "YourToken"}

# initialize a requests.session object with the token headers. All handled by pydap.
my_session = create_session(session_kwargs=session_extra)

```python
ds = open_url(dataset_url, session=my_session, protocol='dap4')
```

```{note}
At this point only the metadata has been downloaded to generate the pydap dataset, and references to each remove variable have been added
```